In [1]:
import pandas as pd
import numpy as np
import re,pyperclip

# Choix

In [2]:
repL4L="/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/L4L/"
repIMM21=repL4L+"IMM21/"

In [3]:
fFreqL4L=repIMM21+"L4L-Frequences.xlsx"
# fControl=repL4L+"vlexique-Sampa.csv"
# fControl=repIMM21+"IMM21-02-X-paradigmes.csv"
fControl=repIMM21+"IMM21-00-X-paradigmes.csv"

In [4]:
strSwim=repIMM21+"IMM21-%02d-X-%s.csv"
strLex=repIMM21+"IMM21-%02d-X-paradigmes.csv"
lSwim=[]
for i in range(3):
    for paradigme in ["paradigmes","Morphomes"]:
        for n in range(2):
            type=paradigme+"-Swim%d"%(n+1)
            for ext in ["","-StemSpace"]:
                type+=ext
                lSwim.append((strSwim%(i,type),strLex%i))
# lSwim[16:]

In [5]:
dfControl=pd.read_csv(fControl,sep=";",encoding="utf8")
# dfContol

In [6]:
dfFreq=pd.read_excel(fFreqL4L)
dictFreq={}

def df2dict(row):
    lemme=row.lemme
    infover=row.infover
    if lemme not in dictFreq:
        dictFreq[lemme]={}
    dictFreq[lemme][infover]=row.freqlog

_=dfFreq.apply(df2dict,axis=1)

In [7]:
# dictFreq

In [8]:
# dfFreq[(dfFreq.lemme.str.contains("^"+"abaisser"+"$"))&(dfFreq.infover=="ai1P")]

In [9]:
cols=dfControl.columns.tolist()[2:]
# cols

In [10]:
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
neutralisations=neutralisationsSUD

bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [11]:
def recoder(chaine,table=neutralise):
    if isinstance(chaine,str):
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif isinstance(chaine,unicode):
        result=chaine.translate(table)
    else:
        result=chaine
    return result

In [12]:
dierese={"j":"ij", "w":"uw","H":"yH","i":"ij","u":"uw","y":"yH"}

In [13]:
def checkFrench(prononciation):
    glide2voyelle={"j":"i","w":"u","H":"y"}
    if prononciation:
        if "," in prononciation:
            prononciations=prononciation.split(",")
            setPrononciations=set()
            for element in prononciations:
                setPrononciations.add(checkFrench(element))
            result=",".join(list(setPrononciations))
        else:
            # print prononciation
            result=recoder(prononciation)
            result=result.replace("Jj","J")
            m=re.match(ur"^(.*[^ieèEaOouy926êôâ])([jwH])$",result)
            if m:
                print ("pb avec un glide final", prononciation)
                voyelle=glide2voyelle[m.group(2)]
                result=m.group(1)+voyelle
            m=re.match(ur"(.*[ptkbdgfsSvzZ][rl])([jwH])(.*)",result)
            if m:
                n=re.search(ur"[ptkbdgfsSvzZ][rl](wa|Hi|wê)",result)
                if not n:
                    glide=m.group(2)
                    result=m.group(1)+dierese[glide]+m.group(3)
            m=re.match(ur"(.*)([iuy])([ieEaOouyêâô].*)",result)
            if m:
                glide=m.group(2)
                result=m.group(1)+dierese[glide]+m.group(3)
            result=result.replace("jj","ij")
    else:
        result=prononciation
    return result

In [14]:
def normalise(forme):
    result=recoder(forme)
    result=checkFrench(result)
    return result        

def compare(row):
    def compareForms(f1,f2):
        if normalise(f1)==normalise(f2):
            lEquals[c]=(f1,f2)
            bEq=True
        else:
            lDifferents[c]=(f1,f2)
            bEq=False
        return bEq
        
    fEquals=0
    fDifferents=0
    fMissings=0
    lexeme=row.lexeme
    if lexeme in lexemes:
        # print
        # print lexeme
        lEquals=equals[lexeme]={}
        lDifferents=differents[lexeme]={}
        lMissings=missings[lexeme]={}
        swimRow=dfSwim[dfSwim.lexeme.str.contains("^"+lexeme+"$")]
        lexRow=dfLex[dfLex.lexeme.str.contains("^"+lexeme+"$")]
        for c in cols:
            if lexRow[c].isnull().any():
                swimRowC=swimRow[c].values[0]
                if row[c]==row[c]:
                    # print c,
                    freq=dictFreq[lexeme][c]
                    if swimRowC==swimRowC:
                        if compareForms(row[c],swimRowC):
                            fEquals+=freq
                        else:
                            fDifferents+=freq
                    else:
                        lMissings[c]=(row[c],swimRowC)
                        fMissings+=freq
        return (lexeme,len(lEquals),len(lDifferents),len(lMissings),fEquals,fDifferents,fMissings)
    else:
        return (lexeme,np.nan,np.nan,np.nan,0,0,0)

In [15]:
dResultats={}
for fSwim,fLex in lSwim[16:]:
    dfSwim=pd.read_csv(fSwim,sep=";",encoding="utf8")
    lexemes=dfSwim["lexeme"].tolist()
    dfLex=pd.read_csv(fLex,sep=";",encoding="utf8")
    equals={}
    differents={}
    missings={}

    fLogEquals=0
    fLogDifferents=0
    fLogMissing=0
    dfResults=pd.DataFrame([[l,e,d,m,fe,fd,fm] for l,e,d,m,fe,fd,fm in dfControl.apply(compare,axis=1).values], columns="lexeme equal different missing freqEqu freqDif freqMis".split(" "))
    nbForms=dfResults["equal different missing".split(" ")].sum().sum()
    nomComparaison=fSwim.split("/")[-1]+" vs "+fControl.split("/")[-1]
    print nomComparaison
    if nbForms>0:
        nbEqual=int(dfResults["equal"].sum())
        nbDifferent=int(dfResults["different"].sum())
        nbMissing=int(dfResults["missing"].sum())
        freqEqual=dfResults["freqEqu"].sum()
        freqDifferent=dfResults["freqDif"].sum()
        freqMissing=dfResults["freqMis"].sum()
        freqTot=freqEqual+freqDifferent+freqMissing
        
        for k,v in differents.iteritems():
            for kk,vv in v.iteritems():
                if "," in vv[0] or "," in vv[1]:
                    set1=set(vv[0].split(","))
                    set2=set(vv[1].split(","))
                    sEquals12=set1&set2
                    equals12=",".join(sEquals12)
                    sMissings12=set1-set2
                    missings12=",".join(sMissings12)
                    sDifferents12=set2-set1
                    differents12=",".join(sDifferents12)
                    # print ",".join([k,kk,"=",equals12,u"Ø",missings12,u"≠",differents12])
                    nbEqual+=len(sEquals12)
                    nbDifferent-=len(sEquals12)
                    nbMissing+=len(sMissings12)
        print "Formes testées : %d"%nbForms
        print "formes identiques :",nbEqual,"=> %2.1f%%"%round(nbEqual/nbForms*100,1)
        print "formes manquantes :",nbMissing,"=> %2.1f%%"%round(nbMissing/nbForms*100,1)
        print "formes différentes :",nbDifferent,"=> %2.1f%%"%round(nbDifferent/nbForms*100,1)
        print "précision : %2.1f%%"%(float(nbEqual)/(nbEqual+nbDifferent)*100)
        print "rappel : %2.1f%%"%(float(nbEqual)/(nbEqual+nbMissing)*100)
        print
        print "somme de l'indicateur de fréquence : %.f"%freqTot
        print "formes identiques Freq : %.f"%freqEqual,"=> %2.1f%%"%round(freqEqual/freqTot*100,1)
        print "formes manquantes Freq : %.f"%freqMissing,"=> %2.1f%%"%round(freqMissing/freqTot*100,1)
        print "formes différentes Freq : %.f"%freqDifferent,"=> %2.1f%%"%round(freqDifferent/freqTot*100,1)
        print "précision Freq: %2.1f%%"%(float(freqEqual)/(freqEqual+freqDifferent)*100)
        print "rappel Freq : %2.1f%%"%(float(freqEqual)/(freqEqual+freqMissing)*100)
        
        dResultats[nomComparaison]=(nbForms, nbEqual, nbMissing, nbDifferent,
                                    float(nbEqual)/(nbEqual+nbDifferent)*100,
                                    float(nbEqual)/(nbEqual+nbMissing)*100,
                                    freqTot,freqEqual,freqMissing,freqDifferent,
                                    float(freqEqual)/(freqEqual+freqDifferent)*100,
                                    float(freqEqual)/(freqEqual+freqMissing)*100
                                   )


IMM21-02-X-paradigmes-Swim1.csv vs IMM21-00-X-paradigmes.csv
Formes testées : 50984
formes identiques : 25480 => 50.0%
formes manquantes : 25498 => 50.0%
formes différentes : 6 => 0.0%
précision : 100.0%
rappel : 50.0%

somme de l'indicateur de fréquence : 90044
formes identiques Freq : 39747 => 44.1%
formes manquantes Freq : 50288 => 55.8%
formes différentes Freq : 9 => 0.0%
précision Freq: 100.0%
rappel Freq : 44.1%
IMM21-02-X-paradigmes-Swim1-StemSpace.csv vs IMM21-00-X-paradigmes.csv
Formes testées : 50984
formes identiques : 28793 => 56.5%
formes manquantes : 22177 => 43.5%
formes différentes : 14 => 0.0%
précision : 100.0%
rappel : 56.5%

somme de l'indicateur de fréquence : 90044
formes identiques Freq : 45457 => 50.5%
formes manquantes Freq : 44547 => 49.5%
formes différentes Freq : 40 => 0.0%
précision Freq: 99.9%
rappel Freq : 50.5%
IMM21-02-X-paradigmes-Swim2.csv vs IMM21-00-X-paradigmes.csv
Formes testées : 50984
formes identiques : 35921 => 70.5%
formes manquantes : 15009 

In [16]:
print "somme de l'indicateur de fréquence : %.f"%freqTot

somme de l'indicateur de fréquence : 90044


In [21]:
dfComparaisonsSwim=pd.DataFrame(dResultats).T
dfComparaisonsSwim.columns="nb identiques manquantes differentes precision rappel freqTot fIdentiques fManquantes fDifferentes fPrecision fRappel".split(" ")
dfComparaisonsSwim.reset_index(inplace=True)
dfComparaisonsSwim["echantillon"]=dfComparaisonsSwim["index"].apply(lambda x: x.split(" vs ")[0].split(".")[0].split("-")[1])
dfComparaisonsSwim["paradigme"]=dfComparaisonsSwim["index"].apply(lambda x: x.split(" vs ")[0].split(".")[0].split("-")[3])
dfComparaisonsSwim["swim"]=dfComparaisonsSwim["index"].apply(lambda x: x.split(" vs ")[0].split(".")[0].split("-")[4])
dfComparaisonsSwim["space"]=dfComparaisonsSwim["index"].apply(lambda x: x.split(" vs ")[0].split(".")[0].split("-")[-1])

In [22]:
dfComparaisonsSwim.loc[dfComparaisonsSwim.paradigme=="Morphomes","paradigme"]="Mor"
dfComparaisonsSwim.loc[dfComparaisonsSwim.paradigme=="paradigmes","paradigme"]="Syn"
dfComparaisonsSwim.loc[dfComparaisonsSwim.space!="StemSpace","space"]="Forms"
dfComparaisonsSwim.loc[dfComparaisonsSwim.space=="StemSpace","space"]="Stems"
dfComparaisonsSwim.set_index("echantillon paradigme swim space".split(" "),inplace=True)
del dfComparaisonsSwim["index"]

In [23]:
dfComparaisonsSwim["precision rappel fPrecision fRappel".split(" ")]

precision     rappel  fPrecision    fRappel
echantillon paradigme swim  space                                             
02          Mor       Swim1 Stems  99.853075  84.083188   99.728230  79.461267
                            Forms  99.936737  74.397567   99.900020  69.960180
                      Swim2 Stems  99.689090  92.075449   99.467183  89.192859
                            Forms  99.775357  86.402311   99.618489  83.203051
            Syn       Swim1 Stems  99.951401  56.490092   99.911530  50.505552
                            Forms  99.976458  49.982345   99.976631  44.145754
                      Swim2 Stems  99.799022  74.131767   99.700367  68.708798
                            Forms  99.849896  70.530139   99.797269  64.956789

In [24]:
dfComparaisonsSwim.to_pickle(repIMM21+"Comparaisons-Swim.pkl")

# Fin

In [26]:
dfComparaisonsSwim02=pd.read_pickle(repIMM21+"Comparaisons-Swim-00-02.pkl")

In [33]:
display (dfComparaisonsSwim["precision rappel fPrecision fRappel".split(" ")])
display(dfComparaisonsSwim02["precision rappel fPrecision fRappel".split(" ")])

precision     rappel  fPrecision    fRappel
echantillon paradigme swim  space                                             
02          Mor       Swim1 Stems  99.853075  84.083188   99.728230  79.461267
                            Forms  99.936737  74.397567   99.900020  69.960180
                      Swim2 Stems  99.689090  92.075449   99.467183  89.192859
                            Forms  99.775357  86.402311   99.618489  83.203051
            Syn       Swim1 Stems  99.951401  56.490092   99.911530  50.505552
                            Forms  99.976458  49.982345   99.976631  44.145754
                      Swim2 Stems  99.799022  74.131767   99.700367  68.708798
                            Forms  99.849896  70.530139   99.797269  64.956789

precision     rappel  fPrecision    fRappel
echantillon paradigme swim  space                                             
00          Mor       Swim1 Stems  99.689624  94.404364   99.714994  93.483281
                            Forms  99.723576  91.539797   99.750552  90.746956
                      Swim2 Stems  99.593477  97.200428   99.596243  96.988857
                            Forms  99.646570  96.097039   99.627185  95.944145
            Syn       Swim1 Stems  99.635009  89.991981   99.669458  89.813570
                            Forms  99.670576  84.199746   99.708267  84.180569
                      Swim2 Stems  99.576063  94.768284   99.568324  94.846803
                            Forms  99.617289  92.792211   99.601045  92.958693

In [42]:
pd.concat([dfComparaisonsSwim.iloc[[2]],dfComparaisonsSwim02.iloc[[2]]]).to_csv(repIMM21+"Comparaisons-Swim-L4L.csv",sep="\t")

In [54]:
print nomComparaison

IMM21-02-X-Morphomes-Swim2-StemSpace.csv vs IMM21-00-X-paradigmes.csv


In [ ]:
missingCells={}
for k,v in missings.iteritems():
    if v:
        # print k,":",
        for kk,vv in v.iteritems():
            # print kk,vv[0],
            if kk not in missingCells: missingCells[kk]=0
            missingCells[kk]+=1
        # print

In [62]:
for cell in sorted(missingCells, key=missingCells.get, reverse=True):
    print(cell, missingCells[cell])
# sorted(missingCells,key=missingCells.get)

(u'inf', 586)
(u'ppMS', 273)
(u'ppFS', 265)
(u'ppMP', 256)
(u'ppFP', 250)
(u'pi3S', 243)
(u'pi1S', 231)
(u'pi2S', 228)
(u'pI2S', 221)
(u'pi3P', 92)
(u'ai3P', 84)
(u'ai1S', 70)
(u'fi3S', 69)
(u'pc3S', 68)
(u'fi3P', 66)
(u'fi2P', 65)
(u'fi2S', 65)
(u'pc2S', 65)
(u'pc3P', 65)
(u'pc1S', 65)
(u'fi1P', 65)
(u'fi1S', 65)
(u'pc2P', 62)
(u'pc1P', 50)
(u'ai3S', 48)
(u'ps3S', 42)
(u'ps2S', 37)
(u'ps1S', 34)
(u'pI1P', 26)
(u'ii1P', 25)
(u'ps2P', 23)
(u'ai1P', 22)
(u'pI2P', 19)
(u'pP', 17)
(u'ii3S', 16)
(u'pi2P', 16)
(u'ii2P', 16)
(u'pi1P', 16)
(u'ii1S', 15)
(u'ii3P', 15)
(u'ii2S', 15)
(u'ps3P', 11)
(u'is3S', 10)
(u'ai2S', 9)
(u'ps1P', 8)
(u'ai2P', 5)
(u'is2P', 4)
(u'is1S', 3)
(u'is1P', 2)
(u'is2S', 1)
(u'is3P', 1)
